In [2]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [20]:
weather

NameError: name 'weather' is not defined

In [4]:
df3 = df3.apply(pd.to_numeric, errors = 'coerce')
df3.describe().iloc[:, 14:]
# 없는 Column = codesum, station_nbr, date

,stnpressure,sealevel,resultspeed,resultdir,avgspeed,nothing,rain,other,store_nbr,station_nbr,item_nbr,units,week7,weekend
count,303363.000000,303363.000000,304362.000000,304362.000000,303696.000000,304362.000000,304362.000000,304362.000000,304362.000000,304362.0,304362.000000,304362.000000,304362.000000,304362.000000
mean,29.282613,30.000977,7.924726,18.748359,9.549781,0.638950,0.352298,0.055799,27.666667,3.0,56.000000,1.700462,3.005470,0.287746
std,0.185680,0.193623,4.545405,9.615155,4.105360,0.480306,0.477687,0.229533,4.988885,0.0,32.041692,15.330650,2.005732,0.452713
min,28.670000,29.400000,0.100000,1.000000,1.600000,0.000000,0.000000,0.000000,21.000000,3.0,1.000000,0.000000,0.000000,0.000000
25%,29.160000,29.870000,4.400000,15.000000,6.500000,0.000000,0.000000,0.000000,21.000000,3.0,28.000000,0.000000,1.000000,0.000000
50%,29.260000,29.970000,7.000000,17.000000,8.800000,1.000000,0.000000,0.000000,29.000000,3.0,56.000000,0.000000,3.000000,0.000000
75%,29.390000,30.110000,11.200000,24.000000,12.300000,1.000000,1.000000,0.000000,33.000000,3.0,84.000000,0.000000,5.000000,1.000000
max,30.010000,30.720000,21.800000,36.000000,22.600000,1.000000,1.000000,1.000000,33.000000,3.0,111.000000,476.000000,6.000000,1.000000


In [7]:
df3_drop_columns = ['date', 'station_nbr', 'codesum']
df3 = df3.drop(columns = df3_drop_columns)

In [9]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df3_columns = df3.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df3_columns:
    if (i == 'resultdir'):
        df3[i].fillna(df3[i].mode()[0], inplace=True)
        print(df3[i].mode()[0])
    else:
        df3[i].fillna(df3[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df3['relative_humility'] = 100*(np.exp((17.625*((df3['dewpoint']-32)/1.8))/(243.04+((df3['dewpoint']-32)/1.8)))/np.exp((17.625*((df3['tavg']-32)/1.8))/(243.04+((df3['tavg']-32)/1.8))))

# 체감온도 계산
df3["windchill"] = 35.74 + 0.6215*df3["tavg"] - 35.75*(df3["avgspeed"]**0.16) + 0.4275*df3["tavg"]*(df3["avgspeed"]**0.16)

# df3 = df3[df3['units'] != 0]

17


In [10]:
model_df3 = sm.OLS.from_formula('np.log1p(units) ~ tmax + tmin + tavg + dewpoint + wetbulb + heat + cool + preciptotal + stnpressure + \
sealevel + resultspeed + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + C(store_nbr) + relative_humility + windchill + 0', data = df3)
result_df3 = model_df3.fit()

print(result_df3.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.433
Model:                            OLS   Adj. R-squared:                  0.433
Method:                 Least Squares   F-statistic:                     1722.
Date:                Thu, 05 Jul 2018   Prob (F-statistic):               0.00
Time:                        20:51:09   Log-Likelihood:            -1.9106e+05
No. Observations:              304362   AIC:                         3.824e+05
Df Residuals:                  304226   BIC:                         3.838e+05
Df Model:                         135                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(nothing)[0]         -0.0009      0

In [11]:
anova_result_df3 = sm.stats.anova_lm(result_df3, typ=2).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df3[anova_result_df3['PR(>F)'] <= 0.05]

,sum_sq,df,F,PR(>F)
C(week7),3.206628,6.0,2.599792,1.607942e-02
C(store_nbr),13.880026,2.0,33.759938,2.187116e-15
C(item_nbr),47777.008516,110.0,2112.844952,0.000000e+00


In [12]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df3.values, i) for i in range(df3.shape[1])]
vif["features"] = df3.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# item_nbr, weekend, week7, store_nbr, resultdir, other, 

,VIF Factor,features
0,1.001656,store_nbr
1,1.006731,item_nbr
2,1.008581,units
3,1.064902,snowfall
4,1.091525,resultdir
5,1.264672,preciptotal
6,1.375097,other
7,2.713263,week7
8,2.728859,weekend
9,2.812192,depart


In [16]:
# item_nbr, weekend, week7, store_nbr, resultdir, other, 
model_df3 = sm.OLS.from_formula('np.log1p(units) ~ C(item_nbr) + C(weekend)  + scale(resultdir) + C(other) +0', data = df3)
result_df3 = model_df3.fit()

print(result_df3.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.433
Model:                            OLS   Adj. R-squared:                  0.433
Method:                 Least Squares   F-statistic:                     2056.
Date:                Thu, 05 Jul 2018   Prob (F-statistic):               0.00
Time:                        20:54:05   Log-Likelihood:            -1.9110e+05
No. Observations:              304362   AIC:                         3.824e+05
Df Residuals:                  304248   BIC:                         3.836e+05
Df Model:                         113                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
C(item_nbr)[1]      -0.0016      0.009  

In [17]:
X3 = df3[['weekend', 'item_nbr', 'store_nbr', 'other', 'resultdir']]
y3 = df3['units']
model3 = LinearRegression()

cv3 = KFold(n_splits=10, shuffle=True, random_state=0)

cross_val_score(model3, X3, y3, scoring="r2", cv=cv3)

array([0.00839901, 0.00853182, 0.00936475, 0.00816546, 0.00837945,
       0.00840137, 0.00865213, 0.00805383, 0.00874061, 0.00755352])